# Trading Performance Reporter - Example Usage

This notebook demonstrates how to use the Trading Performance Reporter package to generate comprehensive reports for your Hyperliquid trading account.

## Features

- **AUM Tracking**: Monitor your Assets Under Management over time
- **Performance Analysis**: Track P&L in both USD and percentage
- **Trade Analysis**: Detailed breakdown of all trades with costs and fees
- **Funding Costs**: Track funding payments for perpetual positions
- **Professional Reports**: Generate HTML reports with visualizations
- **Email Delivery**: Automatically send reports via email

## Setup

First, import the necessary modules:

In [ ]:
import logging
from datetime import datetime, timedelta
from pathlib import Path

from config import load_config
from hyperliquid_reporter.monitoring import HyperliquidMonitor
from hyperliquid_reporter.reporter import HyperliquidReporter

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 1. Load Configuration

Load your Hyperliquid account configuration:

In [ ]:
config = load_config(
    config_path="config_hyperliquid.json",
    testnet=False
)

account_address = config['account_address']
info = config['_hyperliquid_info']

print(f"Account: {account_address[:10]}...{account_address[-8:]}")
print(f"Network: {'Testnet' if config['testnet'] else 'Mainnet'}")

## 2. Initialize Monitor

Create a monitor to fetch account data:

In [ ]:
monitor = HyperliquidMonitor(info=info, address=account_address)
print("Monitor initialized successfully!")

## 3. Get Current Metrics

Fetch current account metrics:

In [ ]:
metrics = monitor.get_current_metrics()
print(f"Account Value: ${metrics.account_value:,.2f}")
print(f"Unrealized P&L: ${metrics.unrealized_pnl:,.2f}")

## 4. Get Account Summary

Get comprehensive account summary including deposits and withdrawals:

In [ ]:
account_summary = monitor.get_account_summary(lookback_days=3650)

print(f"Total Deposits: ${account_summary['total_deposits']:,.2f}")
print(f"Total Withdrawals: ${account_summary['total_withdrawals']:,.2f}")
print(f"Net Deposits: ${account_summary['net_deposits']:,.2f}")
print(f"Current Value: ${account_summary['current_value']:,.2f}")
print(f"Total P&L: ${account_summary['total_pnl']:,.2f}")
print(f"P&L %: {account_summary['pnl_percentage']:.2f}%")
print(f"Spot Value: ${account_summary['spot_value']:,.2f}")
print(f"Perp Value: ${account_summary['perp_value']:,.2f}")

## 5. Initialize Reporter

Create a reporter to generate reports:

In [ ]:
reporter = HyperliquidReporter(monitor=monitor, account_address=account_address)
print("Reporter initialized successfully!")

## 6. Generate AUM Data

Get Assets Under Management over time:

In [ ]:
aum_data = reporter.generate_aum_data(period="allTime")
print(f"AUM Data Points: {len(aum_data)}")
if not aum_data.empty:
    print(f"\nFirst 5 entries:")
    print(aum_data.head())
    print(f"\nLast 5 entries:")
    print(aum_data.tail())

## 7. Generate Performance Data

Get performance metrics (P&L in $ and %):

In [ ]:
performance_data = reporter.generate_performance_data(period="allTime")
print(f"Performance Data Points: {len(performance_data)}")
if not performance_data.empty:
    print(f"\nLast 5 entries:")
    print(performance_data.tail())

## 8. Generate Trade Analysis

Analyze all trades with costs and fees:

In [ ]:
end_time = datetime.now()
start_time = end_time - timedelta(days=30)

trade_analysis = reporter.generate_trade_analysis(
    start_time=start_time,
    end_time=end_time
)

print(f"Total Trades (last 30 days): {len(trade_analysis)}")
if not trade_analysis.empty:
    print(f"\nTrade Summary:")
    print(f"Total Volume: ${trade_analysis['notional'].sum():,.2f}")
    print(f"Total Fees: ${trade_analysis['fee'].sum():,.2f}")
    print(f"Total Net P&L: ${trade_analysis['net_pnl'].sum():,.2f}")
    print(f"\nLast 10 trades:")
    print(trade_analysis.tail(10))

## 9. Generate Funding Analysis

Analyze funding costs for perpetual positions:

In [ ]:
funding_analysis = reporter.generate_funding_analysis(
    start_time=start_time,
    end_time=end_time,
    lookback_days=30
)

print(f"Funding Payments (last 30 days): {len(funding_analysis)}")
if not funding_analysis.empty:
    print(f"\nFunding Summary:")
    print(f"Total Funding: ${funding_analysis['funding_payment'].sum():,.4f}")
    print(f"\nFunding by Coin:")
    funding_by_coin = funding_analysis.groupby('coin')['funding_payment'].sum().sort_values()
    print(funding_by_coin)
    print(f"\nLast 10 funding payments:")
    print(funding_analysis.tail(10))

## 10. Generate Complete Report

Generate all report data at once:

In [ ]:
report_data = reporter.generate_report_data(
    period="allTime",
    start_time=start_time,
    end_time=end_time,
    lookback_days=30
)

print("Report Data Generated!")
print(f"\nSummary Statistics:")
stats = report_data['summary_stats']
for key, value in stats.items():
    if isinstance(value, (int, float)):
        if 'pct' in key:
            print(f"{key}: {value:.2f}%")
        elif isinstance(value, int):
            print(f"{key}: {value:,}")
        else:
            print(f"{key}: ${value:,.2f}")

## 11. Create Visualizations

Generate charts and graphs:

In [ ]:
visualizations = reporter.create_visualizations(
    report_data=report_data,
    output_dir="reports"
)

print(f"Created {len(visualizations)} visualizations:")
for name in visualizations.keys():
    print(f"  - {name}")

## 12. Generate HTML Report

Create a professional HTML report:

In [ ]:
html_content = reporter.generate_html_report(
    report_data=report_data,
    visualizations=visualizations
)

output_path = Path("reports")
output_path.mkdir(parents=True, exist_ok=True)

report_filename = f"trading_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
report_path = output_path / report_filename

with open(report_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"HTML report saved to: {report_path}")
print(f"Report size: {len(html_content):,} bytes")

## 13. Send Report via Email (Optional)

Send the report via email using the mail_it function:

In [ ]:
from mail_it import mail_it

result = mail_it(
    to="silvaac@yahoo.com",
    subj=f"Trading Performance Report - {datetime.now().strftime('%Y-%m-%d')}",
    body=html_content
)

if result == 0:
    print("✓ Report sent successfully!")
else:
    print("✗ Failed to send report")

## Advanced Usage Examples

### Get Portfolio History for Different Periods

In [ ]:
periods = ["day", "week", "month", "allTime"]

for period in periods:
    df = monitor.get_portfolio_dataframe(period=period)
    if not df.empty:
        print(f"\n{period.upper()}:")
        print(f"  Data points: {len(df)}")
        if 'account_value' in df.columns:
            print(f"  Latest AUM: ${df['account_value'].iloc[-1]:,.2f}")
        if 'pnl' in df.columns:
            print(f"  Latest P&L: ${df['pnl'].iloc[-1]:,.2f}")

### Analyze Trade Performance by Coin

In [ ]:
if not trade_analysis.empty:
    print("Trade Performance by Coin:\n")
    
    by_coin = trade_analysis.groupby('coin').agg({
        'notional': 'sum',
        'fee': 'sum',
        'net_pnl': 'sum',
        'coin': 'count'
    }).rename(columns={'coin': 'trade_count'})
    
    by_coin = by_coin.sort_values('notional', ascending=False)
    print(by_coin)

### Calculate Win Rate by Coin

In [ ]:
if not trade_analysis.empty:
    print("Win Rate by Coin:\n")
    
    def calc_win_rate(group):
        total = len(group)
        wins = len(group[group['net_pnl'] > 0])
        return (wins / total * 100) if total > 0 else 0
    
    win_rates = trade_analysis.groupby('coin').apply(calc_win_rate)
    win_rates = win_rates.sort_values(ascending=False)
    
    for coin, rate in win_rates.items():
        print(f"{coin}: {rate:.1f}%")

## Conclusion

This notebook demonstrated all the key features of the Trading Performance Reporter:

1. ✅ Loading configuration and initializing the monitor
2. ✅ Fetching current metrics and account summary
3. ✅ Generating AUM data over time
4. ✅ Analyzing performance (P&L in $ and %)
5. ✅ Analyzing trades with costs and fees
6. ✅ Tracking funding costs for perpetuals
7. ✅ Creating visualizations
8. ✅ Generating professional HTML reports
9. ✅ Sending reports via email

For automated reporting, use the `run_report.py` script:

```bash
python run_report.py
```

You can customize the parameters in `run_report.py` to suit your needs.